In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
import subprocess

print("Python이 현재 인식하고 있는 PATH:")
current_path_env = os.environ.get('PATH', '')
for path_entry in current_path_env.split(os.pathsep):
    print(path_entry)
print("-" * 30)

# 이제 ffmpeg 테스트를 다시 실행
ffmpeg_command = "ffmpeg"
try:
    print(f"Python에서 '{ffmpeg_command}' 명령어 실행 테스트 중...")
    process = subprocess.run(
        [ffmpeg_command, "-version"],
        capture_output=True, text=True, check=True,
        creationflags=subprocess.CREATE_NO_WINDOW
    )
    print(f"'{ffmpeg_command}' 명령어 실행 성공!")
    print("출력 내용:\n", process.stdout[:300])
except FileNotFoundError:
    print(f"오류: '{ffmpeg_command}' 명령어를 Python의 현재 PATH에서 찾을 수 없습니다.")
    print("위에 출력된 'Python이 현재 인식하고 있는 PATH' 목록에 ffmpeg 실행 파일이 있는 디렉토리가 정확히 포함되어 있는지 확인해주세요.")
    print("포함되어 있지 않다면, 환경 변수 설정 및 모든 관련 프로그램/시스템 재시작이 필요합니다.")
except Exception as e:
    print(f"ffmpeg 테스트 중 다른 오류 발생: {e}")

Python이 현재 인식하고 있는 PATH:
c:\Users\Administrator\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-nqLnFQ7l-py3.11\Scripts
C:\Users\Administrator\AppData\Local\pypoetry\Cache\virtualenvs\langchain-kr-nqLnFQ7l-py3.11\Scripts
C:\oraclexe\app\oracle\product\11.2.0\server\bin
C:\Program Files\Eclipse Adoptium\jdk-21.0.6.7-hotspot\bin
C:\Windows\system32
C:\Windows
C:\Windows\System32\Wbem
C:\Windows\System32\WindowsPowerShell\v1.0\
C:\Windows\System32\OpenSSH\
C:\Program Files\nodejs\
C:\Program Files\Git\cmd
C:\Program Files\Docker\Docker\resources\bin
C:\Program Files\dotnet\
C:\Program Files\Graphviz\bin
C:\Users\Administrator\Downloads\ffmpeg-2025-05-05-git-f4e72eb5a3-full_build\ffmpeg-2025-05-05-git-f4e72eb5a3-full_build\bin
C:\Users\Administrator\AppData\Local\Programs\Python\Python313\Scripts\
C:\Users\Administrator\AppData\Local\Programs\Python\Python313\
C:\Users\Administrator\.pyenv\pyenv-win\bin
C:\Users\Administrator\.pyenv\pyenv-wi
------------------------------
Python에서 'f

In [ ]:
import whisper
import os
from openai import OpenAI
import json
import textwrap # 줄바꿈을 위한 모듈
import torch # PyTorch 라이브러리, Whisper 내부에서 사용되며 장치 확인에 사용

# --- OpenAI API 클라이언트 초기화 ---
# 중요: 이 스크립트를 실행하기 전에 OPENAI_API_KEY 환경 변수를 설정하거나,
# 아래 client = OpenAI() 부분을 client = OpenAI(api_key="YOUR_ACTUAL_API_KEY")로 수정해야 합니다.
try:
    client = OpenAI()
    client.models.list() # API 연결 테스트
    print("OpenAI API 클라이언트가 성공적으로 초기화되었습니다.")
except Exception as e:
    print(f"OpenAI API 클라이언트 초기화 중 오류 발생: {e}")
    print("OPENAI_API_KEY 환경 변수가 올바르게 설정되었는지 또는 코드 내에 API 키를 직접 입력했는지 확인해주세요.")
    exit()

LLM_MODEL_FOR_REQUIREMENTS = "gpt-4o" # 또는 "gpt-4-turbo" 권장

# --- Whisper STT 함수 정의 ---
def transcribe_audio_with_whisper(audio_file_path, whisper_model_name="small", language="ko"):
    try:
        print(f"\nWhisper 모델 로딩 중 ('{whisper_model_name}')...")
        model = whisper.load_model(whisper_model_name)
        print("Whisper 모델 로딩 완료.")
    except Exception as e:
        print(f"Whisper 모델 ('{whisper_model_name}') 로딩 중 오류 발생: {e}")
        return None
    print(f"음성 파일 '{audio_file_path}'에서 텍스트 추출 시도 (언어: {language})...")
    try:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        use_fp16 = True if device == "cuda" else False
        print(f"Whisper 추론 장치: {device}, FP16 사용: {use_fp16}")
        result = model.transcribe(audio_file_path, language=language, fp16=use_fp16)
        transcribed_text = result["text"]
        print("🎧 STT 변환 완료 (Whisper)")
        return transcribed_text
    except FileNotFoundError:
        print(f"오류: 음성 파일을 찾을 수 없습니다 - {audio_file_path}")
        print(f"'{audio_file_path}' 파일이 현재 작업 디렉토리 '{os.getcwd()}'에 있는지 확인해주세요.")
        return None
    except Exception as e:
        print(f"음성 파일 변환 중 다른 오류가 발생했습니다: {e}")
        if "ffmpeg" in str(e).lower():
            print("오류 메시지에 'ffmpeg'가 포함되어 있습니다. ffmpeg가 시스템에 올바르게 설치되고 PATH에 등록되어 있는지 확인해주세요.")
        return None

# --- OpenAI 요구사항 추출 에이전트 함수 정의 (프롬프트 대폭 개선 V6) ---
def define_system_prompt_for_requirements_extraction_v6():
    return """
    당신은 극도로 꼼꼼하고 체계적인 시스템 분석가입니다. 당신의 **유일하고 가장 중요한 목표**는 제공된 회의록 텍스트를 **처음부터 마지막 문장까지 한 단어도 놓치지 않고** 읽으면서, 시스템 개발과 관련된 **모든 개별적인 요구사항, 기능 명세, 기술적 결정, 데이터 관련 결정, 작업 할당, 담당자, 일정 목표, 주요 논의점 및 문제점 등을 식별하고 각각을 별도의 JSON 객체로 만들어 리스트로 반환**하는 것입니다.

    **명심하십시오: 당신의 작업은 단순히 회의를 요약하는 것이 아닙니다. 가능한 한 많은 수의 구체적이고 독립적인 항목들을 찾아내어 구조화된 데이터로 만드는 것입니다. 하나의 항목만 찾고 멈춰서는 절대 안 됩니다. 회의록 전체를 샅샅이 뒤져 모든 잠재적 항목을 추출해야 합니다.** 이 회의록은 여러 페이지에 걸친 상세 내용을 담고 있을 수 있으며, 일반적으로 10~30개 또는 그 이상의 항목이 추출될 수 있습니다.

    **출력 규칙 (반드시 준수):**
    1.  전체 응답은 **JSON 객체들의 리스트(배열) 형식**이어야 합니다. `response_format={"type": "json_object"}` 제약 하에, 최상위 응답은 예를 들어 `{"extracted_items": [ { /*항목1*/ }, { /*항목2*/ } ]}` 와 같이 키를 포함한 객체 안에 리스트가 있거나, 또는 리스트 자체가 최상위일 수 있습니다. 어떤 경우든, 최종적으로 파싱될 수 있는 요구사항 목록은 반드시 객체들의 리스트여야 합니다.
    2.  리스트의 각 요소는 아래 정의된 모든 키를 가진 **완전한 JSON 객체**여야 합니다.
    3.  **단 하나의 항목만 식별되더라도, `[ { ... } ]` 와 같이 리스트로 감싸야 합니다.**
    4.  단순 문자열 리스트 등은 절대 허용되지 않습니다.

    각 JSON 객체의 구조:
    - "id": (문자열) "REQ-XXX", "DEC-XXX", "TASK-XXX", "INFO-XXX", "GOAL-XXX" 형식의 고유 ID (XXX는 세자리 숫자).
    - "description": (문자열) 항목에 대한 명확하고 간결한 설명.
    - "type": (문자열) "기능 요구사항", "비기능 요구사항", "데이터 요구사항", "기술 결정", "아키텍처 결정", "작업 할당", "일정 목표", "일반 결정", "주요 정보/맥락" 중 하나.
    - "details": (문자열, 선택) 추가 설명, 논의 배경 등.
    - "participants_or_assignees": (리스트[문자열], 선택) 관련자 또는 담당자.
    - "keywords": (리스트[문자열]) 핵심 키워드.

    **추출 처리 방식 상세 지침:**
    1.  **문서 전체 스캔:** 회의록 첫 문장부터 마지막 문장까지 순차적으로 읽습니다.
    2.  **항목 식별:** 각 문장 또는 관련된 몇 개의 문장 그룹에서 독립된 요구사항, 결정, 작업 등을 나타내는 내용을 찾습니다.
    3.  **개별 객체 생성:** 식별된 각 내용에 대해 위의 JSON 객체 구조에 맞춰 하나의 객체를 생성합니다.
    4.  **모든 항목 포함:** 사소해 보이는 내용일지라도 위의 분류에 해당하면 포함시키십시오.
    5.  **반복:** 문서 끝까지 이 과정을 반복하여 모든 항목을 리스트에 추가합니다. **하나의 항목만 추출하고 멈추는 것은 이 작업의 실패로 간주됩니다.**

    **요청 형식 예시 (이 예시처럼 여러 항목을 찾아내는 것이 중요함을 다시 강조):**
    만약 입력 텍스트의 일부가 다음과 같다면:
    "사용자 맞춤형 관광지 추천, 여행 일정 자동 구성 기능, 지역 기반 식당 명소 추천까지 포함하는 걸로 하죠. SNS 기반 키워드 분석으로 인기 관광지를 추출하면 동적 추천이 가능할 거예요. 하지만 SNS 데이터는 신뢰성이 떨어지고 일관성이 부족합니다."
    기대하는 JSON 출력 (이것은 예시의 일부이며, 실제로는 더 많은 항목이 나올 수 있음):
    ```json
    [
      {
        "id": "REQ-001",
        "description": "사용자 맞춤형 관광지 추천 기능 구현",
        "type": "기능 요구사항",
        "details": "개별 사용자에게 최적화된 관광지를 추천하는 기능을 시스템에 포함한다.",
        "participants_or_assignees": [],
        "keywords": ["사용자 맞춤형", "관광지 추천"]
      },
      {
        "id": "REQ-002",
        "description": "여행 일정 자동 구성 기능 구현",
        "type": "기능 요구사항",
        "details": "사용자의 요구 또는 선호에 따라 여행 일정을 자동으로 생성하고 제안하는 기능을 포함한다.",
        "participants_or_assignees": [],
        "keywords": ["여행 일정 자동 구성"]
      },
      {
        "id": "REQ-003",
        "description": "지역 기반 식당 및 명소 추천 기능 구현",
        "type": "기능 요구사항",
        "details": "현재 위치 또는 특정 지역을 기반으로 주변의 식당과 관광 명소를 추천하는 기능을 포함한다.",
        "participants_or_assignees": [],
        "keywords": ["지역 기반", "식당 추천", "명소 추천"]
      },
      {
        "id": "REQ-004",
        "description": "SNS 키워드 분석을 통한 인기 관광지 동적 추천 기능",
        "type": "기능 요구사항",
        "details": "SNS에서 키워드 분석을 통해 현재 인기 있는 관광지를 추출하고 이를 동적으로 추천한다.",
        "participants_or_assignees": [],
        "keywords": ["SNS", "키워드 분석", "인기 관광지", "동적 추천"]
      },
      {
        "id": "INFO-001",
        "description": "SNS 데이터의 신뢰성 및 일관성 부족 문제점 지적",
        "type": "주요 정보/맥락",
        "details": "SNS 데이터는 동적 추천에 활용될 수 있으나, 신뢰성과 일관성이 부족하다는 문제가 제기됨.",
        "participants_or_assignees": [],
        "keywords": ["SNS 데이터", "신뢰성 부족", "일관성 부족"]
      }
    ]
    ```
    """

def extract_requirements_from_text_via_openai(text_to_analyze, client_instance, model):
    system_prompt = define_system_prompt_for_requirements_extraction_v6() # V6 프롬프트 사용
    user_prompt = f"""
    다음은 회의 음성을 텍스트로 변환한 내용입니다.
    이 내용을 시스템 프롬프트의 지침, 특히 **문서 전체를 스캔하여 모든 개별 항목을 식별하고 각각을 별도의 JSON 객체로 만들어 리스트로 반환하라**는 지침에 따라 매우 주의 깊게 분석해 주십시오. 
    **가장 중요한 것은 가능한 한 많은 세부 항목을 추출하는 것입니다. 단일 요약으로 끝나서는 안 됩니다.**

    --- 회의록 (STT 변환 결과) 시작 ---
    {text_to_analyze}
    --- 회의록 (STT 변환 결과) 끝 ---
    """
    llm_response_content = ""
    try:
        print("\nOpenAI API 호출하여 요구사항 추출 중 (V6 프롬프트)...")
        response = client_instance.chat.completions.create(
            model=model,
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.0 # 온도를 0으로 설정하여 결정론적 행동 유도
        )
        llm_response_content = response.choices[0].message.content
        print("OpenAI API 응답 수신 완료.")
        
        print("\n--- RAW LLM RESPONSE START (DEBUG V6) ---")
        print(llm_response_content) 
        print("--- RAW LLM RESPONSE END (DEBUG V6) ---\n")
        
        parsed_json_outer = json.loads(llm_response_content)
        
        if isinstance(parsed_json_outer, list):
            if not parsed_json_outer or all(isinstance(item, dict) for item in parsed_json_outer):
                return parsed_json_outer
            else:
                print(f"경고 (V6): LLM 응답이 리스트이지만, 내부 항목 중 일부가 JSON 객체(딕셔너리)가 아닙니다. 응답 일부: {llm_response_content[:300]}")
                return []
        elif isinstance(parsed_json_outer, dict):
            core_keys = ["id", "description", "type"] 
            if all(k in parsed_json_outer for k in core_keys):
                print("정보 (V6): LLM이 단일 요구사항 객체를 반환했습니다. 이를 리스트로 감싸서 처리합니다.")
                return [parsed_json_outer] 
            else:
                # 딕셔너리 안에 특정 키 값으로 리스트가 있는지 확인 (예: {"requirements": [...]})
                # 이 부분을 좀 더 명시적으로 키를 지정하도록 유도할 수도 있음 (예: 프롬프트에서 "extracted_items" 키 사용 지시)
                for key_in_dict in parsed_json_outer:
                    value = parsed_json_outer[key_in_dict]
                    if isinstance(value, list) and (not value or all(isinstance(item, dict) for item in value)):
                        print(f"정보 (V6): LLM 응답 딕셔너리 내 키 '{key_in_dict}'에서 요구사항 리스트를 찾았습니다.")
                        return value
                print(f"경고 (V6): LLM 응답이 딕셔너리이지만, 그 자체가 단일 요구사항 객체도 아니고, 내부에 예상된 요구사항 리스트도 찾지 못했습니다. 응답 일부: {llm_response_content[:300]}")
                return []
        else:
            print(f"경고 (V6): LLM 응답이 예상된 리스트 또는 딕셔너리 형식이 아닙니다. 응답 일부: {llm_response_content[:300]}")
            return []
            
    except json.JSONDecodeError as e:
        print(f"LLM 응답 JSON 파싱 오류 (V6): {e}. 응답: {llm_response_content[:500]}...")
        return []
    except Exception as e:
        print(f"OpenAI API 호출 또는 처리 중 예기치 않은 오류 발생 (V6): {e}. 응답: {llm_response_content[:500]}...")
        return []

# --- 메인 실행 로직 ---
if __name__ == "__main__":
    audio_file_to_process = "./docs/meeting.wav"
    example_transcript_for_testing = """
오늘 회의는 JIE 관광 추천 시스템의 주요 기능과 기술 스태크, 그리고 데이터 수집 방식에 대해 논의하는 자리입니다. 사용자 맞춤형 관광지 추천, 여행 일정 자동 구성 기능,지역 기반 식당 명소 추천까지 포함하는 걸로 하죠. SNS 기반 키워드 분석으로 인기 관광지를 추출하면 동적 추천이 가능할 거예요. 하지만 SNS 데이터는 실내성이 떨어지고일관성이 부족합니다. 공식 관광청 API나 트리퍼드바이저 오픈데이터 기반으로 가는 게 더 좋을 것 같아요. 공식 데이터는 갱신주기가 느려요. 요즘 유행하는 트렌드를 반영하지못합니다. 반면 SNS는 감성 분석 정확도 문제와 잘못된 정보도 많습니다. 추천품지를 보장하기 어려워요. 두 분 다 1, 2 있어요. SNS 기반 데이터는 실시간 반영용으로 쓰고추천 점수 산정 시에는 공식 데이터를 가중치 있게 반영하는 방식으로 용합하면 어떨까요? 좋습니다. 두 소스를 조합해서 유연한 추천 시스템을 설계합시다. 협업 필터링 기반 추천이좋을 것 같아요. 사용자 유사도 기반으로 일정을 추천하는 구조로요. 여기에 상황 기반 추천, cbrs 요소도 넣으면 좋겠어요. 날씨나 계절, 여행 목적, 힐링 넥티비티 등에 따라다르게 추천되도록 사용자 입력을 간소화하는 게 중요합니다. 선억의 질문만으로 추천이 나오게 하죠? 맞아요. 그리고 추천 결과는 지도 기반 시각화와 카드 UI로 제공하면 직관적일거예요. 그럼 부기반 SPA로 진행하고 오퍼레이어지로 지도 시각화까지 포함하도록 합시다. 역할 분담 및 일정 설정 데이터 수집 및 정제, 김다운, 최진수 추천 알고리즘 개발,이서윤, 윤지혁 프론트 앤드 및 사용자 UI, 박정호 5월 20일까지 1차 프로토타이 완성, 6월 초 사용자 테스트 목표 추천 결과를 공유 가능한 기능도 넣으면 좋겠습니다. 여행일정 공유 링크 같은 거예요? 좋네요. 친구 초대 기능과 함께 소셜 기능도 일부 고려합시다. 오늘 갈등도 있었지만 서로의 입장을 잘 반영해서 좋은 방향으로 정리된 것 같습니다.모두 수고하셨습니다.
"""
    whisper_model_size = "small"
    output_json_filename = "./output/extracted_audio_requirements_final_v6.json" # 버전 업데이트

    print(f"현재 작업 디렉토리: {os.getcwd()}")

    transcribed_text = None
    if audio_file_to_process and os.path.exists(audio_file_to_process):
        print(f"'{audio_file_to_process}' 파일로 STT 진행...")
        transcribed_text = transcribe_audio_with_whisper(
            audio_file_to_process,
            whisper_model_name=whisper_model_size,
            language="ko"
        )
    
    if not transcribed_text: 
        if audio_file_to_process and not os.path.exists(audio_file_to_process):
             print(f"경고: 음성 파일 '{audio_file_to_process}'을(를) 찾을 수 없습니다. 예시 텍스트로 대체합니다.")
        elif audio_file_to_process:
             print(f"경고: '{audio_file_to_process}'에서 STT 실패 또는 빈 결과. 예시 텍스트로 대체합니다.")
        else:
            print("STT를 건너뛰고 제공된 예시 텍스트로 요구사항 추출을 테스트합니다.")
        transcribed_text = example_transcript_for_testing.strip()

    if transcribed_text:
        print("\n--- 입력 텍스트 (STT 결과 또는 예시) ---")
        wrapped_stt_output = textwrap.fill(transcribed_text.strip(), width=100, subsequent_indent="  ")
        print(wrapped_stt_output)
        print("----------------------------------------")

        print("\nSTT 변환 텍스트를 기반으로 요구사항 추출을 시작합니다...")
        extracted_requirements = extract_requirements_from_text_via_openai(
            transcribed_text,
            client,
            LLM_MODEL_FOR_REQUIREMENTS
        )

        if extracted_requirements: 
            if isinstance(extracted_requirements, list) and \
               (not extracted_requirements or all(isinstance(item, dict) for item in extracted_requirements)):
                print(f"\n--- OpenAI가 추출한 요구사항 (총 {len(extracted_requirements)}개) ---")
                print(json.dumps(extracted_requirements, ensure_ascii=False, indent=2))

                try:
                    with open(output_json_filename, 'w', encoding='utf-8') as f:
                        json.dump(extracted_requirements, f, ensure_ascii=False, indent=4)
                    print(f"\n추출된 요구사항이 '{output_json_filename}' 파일에 성공적으로 저장되었습니다.")
                except IOError as e:
                    print(f"파일 저장 중 오류 발생 ('{output_json_filename}'): {e}")
            else:
                print("\n오류: LLM이 최종적으로 예상된 JSON 객체 리스트 형식이 아닌 다른 것을 반환했습니다.")
                print("반환된 내용:", extracted_requirements)
        else:
            print("\nOpenAI LLM으로부터 유효한 요구사항을 추출하지 못했습니다 (결과가 비어있거나 내부 처리 실패).")
            print("위의 'RAW LLM RESPONSE (DEBUG V6)' 내용을 다시 확인하여 LLM이 어떤 응답을 보냈는지 살펴보세요.")
    else:
        print("\n입력 텍스트(음성 파일 또는 예시)를 처리하지 못했기 때문에 요구사항 추출을 진행할 수 없습니다.")

1. PDF 전체 텍스트 추출 중...
   PDF 전체 텍스트 추출 완료. 총 54 페이지, 전체 텍스트 길이: 49066자.

2. 목차(ToC) 원문 텍스트 추출 중 (지정 페이지: 2, 3)...
   목차 원문 텍스트 추출 완료 (길이: 8670자).

3. LLM을 사용하여 목차(ToC) 파싱 중...
   LLM 목차 파싱 완료. 49개의 목차 항목 식별.

4. 파싱된 목차에서 주요 요구사항 섹션 범위 식별 중...
키워드 기반 섹션 식별 실패. 'is_requirement_related=True' 플래그로 섹션 식별 시도...
LLM 'is_requirement_related' 플래그 기반: '1. 제안 요청 개요' 섹션 (페이지 1-4) 식별
LLM 'is_requirement_related' 플래그 기반: '1.1. JBANK 시스템 재구축 추진 배경' 섹션 (페이지 1-4) 식별
LLM 'is_requirement_related' 플래그 기반: '1.2. JBANK 시스템 재구축 추진 목표' 섹션 (페이지 1-4) 식별
LLM 'is_requirement_related' 플래그 기반: '1.3. JBANK 시스템 재구축 사업 범위' 섹션 (페이지 2-4) 식별
LLM 'is_requirement_related' 플래그 기반: '4. 프로젝트 구축 범위 및 제안 요청 상세' 섹션 (페이지 9-11) 식별
LLM 'is_requirement_related' 플래그 기반: '4.1.1. JBANK 시스템 UI/UX 표준 체계 마련 : 1.3. JBANK 시스템 재구축 사업 범위 참조' 섹션 (페이지 9-11) 식별
LLM 'is_requirement_related' 플래그 기반: '4.1.2. JBANK 시스템 GRC 체계 마련(확인필요)' 섹션 (페이지 9-11) 식별
LLM 'is_requirement_related' 플래그 기반: '4.1.3. JBANK 시스템 개발 프레임워크 도입(확인필요)' 섹션 (페이지 9-11) 식별
LLM 'is